### Create Embedding for the visualizer

This notebook shows how to create embeddings for the PCA / tSNE visualizer in tensorboard. We use a trained VGG16 network and feed CIFAR images through it.


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.misc
import pickle
%matplotlib inline
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.__version__

'1.0.1'

### Loading of the data

The 32x32 CIFAR images we use here can be obtained from: http://www.cs.toronto.edu/~kriz/cifar.html 

In [2]:
path = './cifar-10-batches-py/test_batch'
f = open(path, 'rb')
data = pickle.load(f, encoding='bytes')
imgs = data[b'data']
imgs = np.transpose(np.reshape(imgs,(-1,32,32,3), order='F'),axes=(0,2,1,3)) #order batch,x,y,color
y = np.asarray(data[b'labels'], dtype='uint8')
y.shape, imgs.shape

((10000,), (10000, 32, 32, 3))

### Defining the models

We use a trained VGG16 network. As a first step we download the VGG16 weights (filename vgg_16.tar.gz) from [here](https://github.com/tensorflow/models/blob/master/slim/README.md#Pretrained) and extract it. After extraction you should get a file named vgg_16.ckpt.

For more details refere to http://randomthoughtsonr.blogspot.de/2016/11/image-classification-in-r-using-trained.html

In [3]:
tf.reset_default_graph()
images = tf.placeholder(tf.float32, [None, None, None, 3])
imgs_scaled = tf.image.resize_images(images, (224,224))
slim.nets.vgg.vgg_16(imgs_scaled, is_training=False)
variables_to_restore = slim.get_variables_to_restore()
print('Number of variables to restore {}'.format(len(variables_to_restore)))

Number of variables to restore 32


In [4]:
init_assign_op, init_feed_dict = slim.assign_from_checkpoint('./vgg_16.ckpt', variables_to_restore)

In [5]:
sess = tf.Session()
sess.run(init_assign_op, init_feed_dict)

In [6]:
g = tf.get_default_graph()
feed = g.get_tensor_by_name('Placeholder:0')
fetch = g.get_tensor_by_name('vgg_16/fc6/BiasAdd:0')

# Feeding 3 images through the net just for testing
feed_vals = imgs[0:3]
res = sess.run(fetch, feed_dict={feed:feed_vals})
np.shape(feed_vals), res.shape

((3, 32, 32, 3), (3, 1, 1, 4096))

### Getting the embedding

We now create a $N \times p$ matrix holding the embeddings, for the N images. For simplycify, we do this by feeding the images one after another through the network (of course we could have also used minibatches). We store this matrix in `EMB` to use it later.

In [7]:
N = imgs.shape[0]
N = 1000 #For testing 
p = res.shape[3]

EMB = np.zeros((N, p), dtype='float32')
for i in range(N): #Of course you could do mini-batches
    EMB[i] = sess.run(fetch, feed_dict={feed: imgs[i:i+1,:]})
    if (i % 50 == 0 or i < 5):
        print(i)

0
1
2
3
4
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


### Writing out the embedding matrix
We now write out the embedding matrix in the proper format needed for the visualizer. If you don't want images or meta data, just comment out the respective parts.

In [8]:
LOG_DIR = 'c:\\Users\\Main\\Documents\\Anaconda\\MY_TESTS\\TensorBoard\\CIFAR\\tmp\\dumm\\'

# The embedding variable, which needs to be stored
# Note this must a Variable not a Tensor!
embedding_var = tf.Variable(EMB,  name='Embedding_of_fc6')
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Comment out if you don't have metadata
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# Comment out if you don't want sprites
embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
embedding.sprite.single_image_dim.extend([imgs.shape[1], imgs.shape[1]])

projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(LOG_DIR, 'model2.ckpt'), 1)

'./tmp/dumm\\model2.ckpt-1'

### Creation of the meta data (labels)

In [9]:
names = ['plane','auto','bird','cat','deer','dog','frog','horse','ship','truck']
metadata_file = open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w')
metadata_file.write('Name\tClass\n')
for i in range(N):
    metadata_file.write('%06d\t%s\n' % (i, names[y[i]]))
metadata_file.close()

### Creation of the images

In [10]:
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [12]:
import scipy.misc

In [13]:
sprite = images_to_sprite(imgs)
scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)